In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/stock-prediction-dataset-adp/ADP-Data.csv


# load the dataset

In [15]:
data_dir = '/kaggle/input/stock-prediction-dataset-adp/ADP-Data.csv'
# to ingnore the first col use [index_col=0]
data = pd.read_csv(data_dir,index_col=0)
data[:10]

,Date,High,Low,Open,Close,Volume,Adj Close
0,1980-03-17,0.927141,0.908554,0.0,0.917847,1158040.1,0.520855
1,1980-03-18,0.930239,0.914749,0.0,0.927141,1622063.1,0.525637
2,1980-03-19,0.942631,0.927141,0.0,0.942631,1367859.1,0.533608
3,1980-03-20,0.961218,0.939533,0.0,0.939533,3046412.1,0.532014
4,1980-03-21,0.948826,0.936435,0.0,0.936435,1020851.1,0.530420
5,1980-03-24,0.942631,0.930239,0.0,0.930239,778752.1,0.527231
6,1980-03-25,0.942631,0.927141,0.0,0.930239,944186.1,0.527231
7,1980-03-26,0.942631,0.927141,0.0,0.942631,2332220.1,0.533608
8,1980-03-27,0.930239,0.911652,0.0,0.911652,770682.1,0.517667
9,1980-03-28,0.942631,0.930239,0.0,0.936435,459988.1,0.530420


# a little data analysis 

In [17]:
# check if it is null data 
data.isnull().sum()

Date         0
High         0
Low          0
Open         0
Close        0
Volume       0
Adj Close    0
dtype: int64

需要做一个按照时间为顺序，最高价和最低价之间的差距的折线图
https://matplotlib.org/stable/gallery/lines_bars_and_markers/cohere.html#sphx-glr-gallery-lines-bars-and-markers-cohere-py

In [19]:
import matplotplt.pyplot as plt 
plt.plot()
plt.

TypeError: '<' not supported between instances of 'str' and 'int'